# Apresentação:

    A ideia aqui é construir um modelo de LLM implementando o RAG,
    Retrieval Augmented Generation. Visando essa efervecência de
    concursos públicos, a ideia aqui é criar um modelo especializado
    em direito, vamos começar com direito constitucional.

Nota:

    As ideias tratadas aqui que já tenham sido tratadas antes, não
    serão reescritas aqui, então, caso haja interesse, busque os códigos
    anteriores no repositorio do github.

In [ ]:
# Instalações Necessárias:
!pip install openai
!pip install cohere
!pip install tiktoken
!pip install vectordb2
!pip install tqdm
!pip install langchain
!pip install transformers[torch]

# Implementando o RAG:

    RAG é Retrieval Augmented Generation é um método de melhoria de um
    LLM baseado no que eu vou chamar aqui de "revalidação". Antes do
    modelo dar a resposta, a ideia é ele validar essa resposta num conjunto
    de documentos especializados.

    É como se, antes do modelo dar a resposta, ele fosse checar a resposta
    com um especialista.

**Preparando o texto base**

In [ ]:
# MRPT - fast nearest neighbor search with random projection
!pip install git+http://github.com/vioshyvo/mrpt/

  Cloning http://github.com/vioshyvo/mrpt/ to /tmp/pip-req-build-v8ks6qlb
  Running command git clone --filter=blob:none --quiet http://github.com/vioshyvo/mrpt/ /tmp/pip-req-build-v8ks6qlb
  Resolved http://github.com/vioshyvo/mrpt/ to commit 88cc6f40782ca0f8de7491279766ded01d767861
  Preparing metadata (setup.py) ... done
  Created wheel for mrpt: filename=mrpt-1.0-cp310-cp310-linux_x86_64.whl size=1561746 sha256=bbc28dc9bc2c0595047c4b35e4ca53c84995ff017c8689fee8de634aea4f9df6
  Stored in directory: /tmp/pip-ephem-wheel-cache-tp38aw78/wheels/61/f0/46/8fd08e2aa4be121079dc3ef4634352680489c4028bdb57e4de
Successfully built mrpt


In [ ]:
# Bibliotecas para requisição e tratamento do texto base:
import re
import requests

In [ ]:
# Importando o Texto bruto:
bruto_text = requests.get('https://raw.githubusercontent.com/abjur/constituicao/main/CONSTITUICAO.md').text

# Otimização do Texto:
padrao_capitulo = r'^##\s+(.*)$' # A ideia é pegar tudo que vem depois do ##
sections = re.split(padrao_capitulo, bruto_text, flags=re.MULTILINE)
sections = [section.strip() for section in sections[1:]]

**Indexando os dados no VectorDB**

In [ ]:
from tqdm import tqdm
from vectordb import Memory
from langchain.text_splitter import MarkdownHeaderTextSplitter

ImportError: cannot import name 'Memory' from 'vectordb' (/usr/local/lib/python3.10/dist-packages/vectordb/__init__.py)

In [ ]:
# Instanciando o SGBD:
memory = Memory(chunking_strategy={'mode':'sliding_window',
                                   'window_size':128, # Chunk_size
                                   'overlap':8 # Chunk Overlap / Janela Deslizante
                                   })

In [ ]:
# Tokenizando as strings:
padrao_capitulo = [("##","Capitulo")]
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=padrao_capitulo)
sections = markdown_splitter.split_text(bruto_text)

In [ ]:
# Indexando os dados:
for i in tqdm(range(0,len(sections))):
  capitulo = sections[i].metadata
  texto = sections[i].page_content

  metadata = {'capitulo':capitulo,
              'origem':'Constituição Federal'}

  memory.save(texto,metadata)

In [ ]:
# Teste de Pesquisa:
memory.search('direitos dos trabalhadores', top_n=5)

# Construção do LLM:

In [ ]:
#import nltk
#import pathlib

In [ ]:
#nltk.download('machado', download_dir='../data/gpt/raw')
#
#!mv ../data/gpt/raw/corpora/machado.zip ../data/gpt/raw/machado.zip
#!unzip ../data/gpt/raw/machado.zip -d ../data/gpt/raw
#!rm -R ../data/gpt/raw/corpora
#!rm ../data/gpt/raw/machado.zip

In [ ]:
## Abrindo o arquivo para extrair o conteudo:
#with open('../data/gpt/raw/machado/contos/macn001.txt', 'r', encoding='iso-8859-1') as f:
#    lines = f.read().splitlines()
#
## Juntando todas os livros
#text = []
#for text_path in sorted(pathlib.Path("../data/gpt/raw/machado/").rglob("*.txt")):
#    with open(text_path, 'r', encoding='iso-8859-1') as f:
#        lines = f.read().splitlines()
#    text += lines
#
#text = " ".join(text)
#
#with open("../data/raw/machado-all.txt", "w") as output:
#     output.write(text)

In [ ]:
#precisamos de um corpus bem robusto, porém não tão gigante
!wget -O ./sample_data/crepusculoDosIdolos.txt https://raw.githubusercontent.com/mfmarlonferrari/NietzscheLLM/main/crepusculoDosIdolos.txt

--2024-02-04 22:51:58--  https://raw.githubusercontent.com/mfmarlonferrari/NietzscheLLM/main/crepusculoDosIdolos.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162098 (158K) [text/plain]
Saving to: ‘./sample_data/crepusculoDosIdolos.txt’

./sample_data/crepu 100%[===================>] 158.30K  --.-KB/s    in 0.03s   

2024-02-04 22:51:58 (5.45 MB/s) - ‘./sample_data/crepusculoDosIdolos.txt’ saved [162098/162098]



In [ ]:
PATH = './sample_data/'
dados_treino = 'crepusculoDosIdolos.txt'

In [ ]:
# Criando o tonkenizer baseado no algoritmo BPE
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
from transformers import (RobertaTokenizer,RobertaForMaskedLM,
                          RobertaConfig,LineByLineTextDataset,
                          DataCollatorForLanguageModeling,
                          Trainer, TrainingArguments,pipeline)

In [ ]:
# Instanciando o modelo:
tokenizer = ByteLevelBPETokenizer()

tokenizer.train(files=[PATH+dados_treino],vocab_size=52_000,
                min_frequency=2, special_tokens=[
                    "<s>",
                    "<pad>",
                    "</s>",
                    "<unk>",
                    "<mask>"])

In [ ]:
# Salvando modelo:
!rm -r ./sample_data/RAW_MODEL
!mkdir ./sample_data/RAW_MODEL
tokenizer.save_model(PATH+"RAW_MODEL")

rm: cannot remove './sample_data/RAW_MODEL': No such file or directory


['./sample_data/RAW_MODEL/vocab.json', './sample_data/RAW_MODEL/merges.txt']

In [ ]:
# Instanciando modelo:
tokenizer = ByteLevelBPETokenizer(
    PATH+'RAW_MODEL'+'/vocab.json',
    PATH+'RAW_MODEL'+'/merges.txt')

tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s",tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>"))
)
tokenizer.enable_truncation(max_length=512)

In [ ]:
# Instanciando o Tokenizer do LLM escolhido:
tokenizer = RobertaTokenizer.from_pretrained(PATH+'RAW_MODEL',max_len=512)

In [ ]:
 # Configurando o Transformer
 config = RobertaConfig(
     vocab_size=52_000,
     max_position_embeddings = 512,
     num_attention_heads = 12,
     num_hidden_layers = 6,
     type_vocab_size = 1)

In [ ]:
# Instanciando Modelo:
model = RobertaForMaskedLM(config=config)

In [ ]:
# Tokenizando os dados de Treino:
dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path=PATH+dados_treino,
    block_size=128
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
# Teste:
tokenizer.decode(dataset.examples[7]['input_ids'])

'<s>na escola bélica da vida — o que não me faz morrer me torna mais forte.</s>'

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.1)

In [ ]:
# Elementos do treinamento:
training_args = TrainingArguments(
    output_dir=PATH+'RAW_MODEL',
    overwrite_output_dir=True,
    num_train_epochs=1200,
    per_device_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True)

# Treinanmento:
Trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset)

In [ ]:
# Fit:
Trainer.train()

Step,Training Loss
500,6.550400
1000,5.226500
1500,4.217900
2000,3.230900
2500,2.382700
3000,1.717100
3500,1.235900
4000,0.919400
4500,0.749400


TrainOutput(global_step=4800, training_loss=2.7748859246571858, metrics={'train_runtime': 5263.4105, 'train_samples_per_second': 51.526, 'train_steps_per_second': 0.912, 'total_flos': 8992119855513600.0, 'train_loss': 2.7748859246571858, 'epoch': 1200.0})

In [ ]:
Trainer.save_model(PATH+'RAW_MODEL')

In [ ]:
# Testando o LLM:
fill_mask = pipeline(
    "fill-mask",
    model=PATH+'RAW_MODEL',
    tokenizer=PATH+'RAW_MODEL')

# Primeiro teste:
texto = 'Digo que o amor é <mask>'
fill_mask(texto)

[{'score': 0.05255282297730446,
  'token': 300,
  'token_str': ' um',
  'sequence': 'Digo que o amor é um'},
 {'score': 0.040366280823946,
  'token': 271,
  'token_str': ' de',
  'sequence': 'Digo que o amor é de'},
 {'score': 0.028334809467196465,
  'token': 961,
  'token_str': ' humanidade',
  'sequence': 'Digo que o amor é humanidade'},
 {'score': 0.0263307336717844,
  'token': 414,
  'token_str': ' —',
  'sequence': 'Digo que o amor é —'},
 {'score': 0.01727619580924511,
  'token': 575,
  'token_str': ' há',
  'sequence': 'Digo que o amor é há'}]

In [ ]:
# Segundo teste:
texto = 'O <mask> da moral: basear na lógica dos fracos'
fill_mask(texto)

[{'score': 0.11579568684101105,
  'token': 800,
  'token_str': ' erro',
  'sequence': 'O erro da moral: basear na lógica dos fracos'},
 {'score': 0.033164724707603455,
  'token': 480,
  'token_str': ' "',
  'sequence': 'O " da moral: basear na lógica dos fracos'},
 {'score': 0.027672506868839264,
  'token': 459,
  'token_str': ' moral',
  'sequence': 'O moral da moral: basear na lógica dos fracos'},
 {'score': 0.015147262252867222,
  'token': 338,
  'token_str': ' por',
  'sequence': 'O por da moral: basear na lógica dos fracos'},
 {'score': 0.013160984963178635,
  'token': 389,
  'token_str': 'res',
  'sequence': 'Ores da moral: basear na lógica dos fracos'}]

In [ ]:
from transformers import RobertaForCausalLM

# Carregando o modelo treinado
modelo = RobertaForCausalLM.from_pretrained(PATH+'RAW_MODEL')
tokenizer = RobertaTokenizer.from_pretrained(PATH+'RAW_MODEL')

If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`


In [ ]:
# Função para gerar respostas
def gerar_resposta(texto_entrada, max_length=100, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95):
    # Tokenizando a entrada
    tokens = tokenizer.encode(texto_entrada, return_tensors="pt")

    # Obtendo a saída do modelo
    output = modelo.generate(tokens, max_length=max_length, num_beams=num_beams, no_repeat_ngram_size=no_repeat_ngram_size, top_k=top_k, top_p=top_p)

    # Decodificando a saída
    resposta = tokenizer.decode(output[0], skip_special_tokens=True)

    return resposta

In [ ]:
# Exemplo de conversa
pergunta = "Qual é o papel do presidente na Constituição Federal?"
resposta = gerar_resposta(pergunta, max_length=200, num_beams=8, no_repeat_ngram_size=3, top_k=100, top_p=0.95)
resposta

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:396: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `100` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


'Qual é o papel do presidente na Constituição Federal?;;; são são são — — — todas todas ca ca............ caso caso caso se se se dos dos vida vida vida pecado vida vida dos dos dos tão tão tão que que que pessoales pois pois pois sentido sentido sentidodosdosdos da da darrr foi foi foi um um um todo todo todo as as as entre entre seu seu seuororor pode o o o exemplo exemplo exemplo para para para ao ao ao não não nãodosdos isto isto isto às às às até até até caso caso com com com seus seus seus mas mas mas e e e quer quer quer nem nem nem tem tem tem parece parece parece instintos instintos instintos fala fala fala os os os na na na à à à na na quando quando quando uma uma uma quem quem quem-- forma forma fim fim fim meio meio meiosesese,,'

In [ ]:
# Exemplo de conversa
pergunta = "Qual é o erro da moral?"
resposta = gerar_resposta(pergunta, max_length=200, num_beams=8, no_repeat_ngram_size=3, top_k=100, top_p=0.95)
resposta

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:396: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `100` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


'Qual é o erro da moral? efeito efeito efeitotototo seu seu seu... muito muito muito " " "iii contra contra contra da da da pouco pouco pouco linguagem linguagem linguagem até até caso caso casoina dessa dessa dessa::: necessário necessário necessário quando quando quando ainda ainda ainda razão razão razão moral moral moral esse esse esse humanidade humanidade humanidade".". estado estado estado na na na pela um um um tudo tudo tudo isso isso isso a a a estado estado".".".,,, dos dos dos castraçãoismoismoismo ele ele qualquer qualquer qualquer toda toda toda forma forma forma representa representa representa espécie espécie espécie uma uma uma do do do e e e aos aos aos o o orara em em emdedede há há há--- no no no que que que sem sem sem porém porém porém não não não mais mais mais coisas coisas coisasááá de de de diz diz diz homem homem homem arte arte artemomomo,,'

In [ ]:
# Função para recuperar documentos relevantes do VectorDB
def recuperar_documentos_relevantes(pergunta, num_documentos=5):
    documentos_relevantes = memory.search(pergunta, top_n=num_documentos)
    return [{'text': doc.get('text', doc.get('page_content', '')), 'origem': doc['metadata']['origem']} for doc in documentos_relevantes]

In [ ]:
# Função para gerar respostas com RAG
def gerar_resposta_rag(pergunta, num_documentos=5, max_length=100, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95):
    # Recuperar documentos relevantes
    documentos_relevantes = recuperar_documentos_relevantes(pergunta, num_documentos)

    # Concatenar documentos relevantes para formar o contexto
    contexto = " ".join([doc['text'] for doc in documentos_relevantes])

    # Adicionar a pergunta ao contexto
    entrada_modelo = contexto + " Pergunta: " + pergunta

    # Tokenizar a entrada
    tokens = tokenizer.encode(entrada_modelo, return_tensors="pt")

    # Obtendo a saída do modelo
    output = modelo.generate(tokens, max_length=max_length, num_beams=num_beams, no_repeat_ngram_size=no_repeat_ngram_size, top_k=top_k, top_p=top_p)

    # Decodificando a saída
    resposta = tokenizer.decode(output[0], skip_special_tokens=True)

    return resposta

In [ ]:
# Primeiro exemplo
pergunta = "Qual é o papel do presidente na Constituição Federal?"
resposta_rag = gerar_resposta_rag(pergunta)
resposta_rag

'     Pergunta: Qual é o papel do presidente na Constituição Federal? " "..).).;; ideia ideia se setata e e primeiro primeiro um um vida vida dos dos tão tão tanto tanto in in não não mais mais de de humanidade humanidade vez vez sua suatotodosdos-- forma formaii a a maneira maneira os os na na que que'

In [ ]:
# Segundo exemplo
pergunta = "Qual é o erro da moral?"
resposta_rag = gerar_resposta_rag(pergunta)
resposta_rag

'     Pergunta: Qual é o erro da moral? " " decadência decadênciaaa-- forma.. suas suas?? a seus seus humanidade humanidade homens homens até até caso caso se se coisas coisas ser sertt;; guerra que que cristã cristã diz diz uma umavv a a isso isso tempo tempo esta os os são são!!; nada nada um um às àsuu o o todas todas fazer fazeria,,'